In [11]:
from rikai.spark.utils import init_spark_session
from rikai.spark.functions import init

spark = init_spark_session(
    dict(
        [
            (
                "spark.rikai.sql.ml.registry.torchhub.impl",
                "ai.eto.rikai.sql.model.torchhub.TorchHubRegistry",
            )
        ]
    )
)

init(spark)

22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function area replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function box2d replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function box2d_from_center replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function box2d_from_top_left replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function copy replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function crop replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function to_image replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function image_copy replaced a previously registered function.
22/01/14 17:52:19 WARN SimpleFunctionRegistry: The function numpy_to_image replaced a previously 

In [21]:
df = spark.sql("select to_image('https://i.scdn.co/image/ab67616d0000b273466def3ce70d94dcacb13c8d') as image")
df.first().image

Image(uri=https://i.scdn.co/image/ab67616d0000b273466def3ce70d94dcacb13c8d)

In [13]:
spark.sql("""
CREATE MODEL yolov5s
FLAVOR yolov5 
OPTIONS (device="cpu", batch_size=32)
USING "torchhub:///ultralytics/yolov5:v6.0/yolov5s";
""")

DataFrame[]

In [23]:
df = spark.sql("""
from (
  select to_image('https://i.scdn.co/image/ab67616d0000b273466def3ce70d94dcacb13c8d') as image
)
select *, ML_PREDICT(yolov5s, image) as pred
""")
row = df.first()
row

Using cache found in /Users/da/.cache/torch/hub/ultralytics_yolov5_v6.0
YOLOv5 🚀 2021-12-29 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
/Users/da/.pyenv/versions/rikai-yolov5/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Row(image=Image(uri=https://i.scdn.co/image/ab67616d0000b273466def3ce70d94dcacb13c8d), pred=[Row(box=Box2d(xmin=167.80673217773438, ymin=311.37408447265625, xmax=199.79217529296875, ymax=333.45159912109375), score=0.8015939593315125, label_id=None), Row(box=Box2d(xmin=262.6680603027344, ymin=176.5178680419922, xmax=360.4544982910156, ymax=550.5360107421875), score=0.7627562880516052, label_id=None), Row(box=Box2d(xmin=310.24041748046875, ymin=169.75477600097656, xmax=440.298828125, ymax=576.0487670898438), score=0.7091681361198425, label_id=None), Row(box=Box2d(xmin=118.68901824951172, ymin=315.7107849121094, xmax=130.5008544921875, ymax=325.4837951660156), score=0.6010026335716248, label_id=None), Row(box=Box2d(xmin=214.30221557617188, ymin=304.1684875488281, xmax=224.65121459960938, ymax=332.8539733886719), score=0.5378967523574829, label_id=None), Row(box=Box2d(xmin=81.23269653320312, ymin=307.76776123046875, xmax=105.72154235839844, ymax=332.25567626953125), score=0.401571065187454

In [25]:
row.image | row.pred[1].box

Draw(Box2d(xmin=262.6680603027344, ymin=176.5178680419922, xmax=360.4544982910156, ymax=550.5360107421875))

In [26]:
row.image | row.pred[2].box

Draw(Box2d(xmin=310.24041748046875, ymin=169.75477600097656, xmax=440.298828125, ymax=576.0487670898438))

In [27]:
row.image | row.pred[3].box

Draw(Box2d(xmin=118.68901824951172, ymin=315.7107849121094, xmax=130.5008544921875, ymax=325.4837951660156))